In [1]:
import os
os.getcwd()

'/home/onyxia/work/hackathon-un-2022/notebooks'

In [8]:
os.chdir("..")
import utils.functions as fc
import utils.indicators as indic
import pandas as pd
import s3fs

In [3]:
from importlib import reload
fc = reload(fc)
indic = reload(indic)

In [10]:
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://minio.lab.sspcloud.fr'}
)

In [14]:
bucket = 'projet-hackathon-un-2022'
data = 'AIS/arrival_counts_april_19.csv'

arrivals_19 = pd.read_csv(
    fs.open(f'{bucket}/{data}',
            mode='rb')
).iloc[:, 1:]

In [15]:
arrivals_19

,port,origin,ShipTypeLevel3,count,GrossTonnage,NetTonnage
0,"""BDR$6.)!Y^5;FNC4(G",SAMSUN,General Cargo,1,2598.000000,1163.000000
1,-,-C?M,Fish Catching,1,721.000000,216.000000
2,-,KRYM-KAVKAZ,General Cargo,4,57813.192782,30764.864637
3,-C?M,-,Fish Catching,1,721.000000,216.000000
4,-TSINGELI,-,General Cargo,1,14453.298196,7691.216159
...,...,...,...,...,...,...
3380,ZONGULDAK,ORDER,General Cargo,1,2160.000000,1243.000000
3381,ZONGULDAK,YEISK,General Cargo,1,14453.298196,7691.216159
3382,ZONGULDAK TURKEY,TAGANROG RUSSIA,General Cargo,1,6483.000000,3264.000000
3383,ZPR,KHERSON,General Cargo,2,4125.000000,1486.000000


On veut identifier les ports d'arrivée

In [22]:
port_data = pd.read_csv('https://msi.nga.mil/api/publications/download?type=view&key=16920959/SFH00000/UpdatedPub150.csv')
port_data = port_data.loc[:, ["Main Port Name", "Country Code"]]
port_data["Main Port Name"] = port_data["Main Port Name"].str.upper()
port_data = port_data.rename({"Main Port Name" : "port", "Country Code": "country"}, axis=1)

In [24]:
port_data.port.nunique()

2828

In [25]:
port_data.shape

(2880, 2)

In [26]:
port_data.drop_duplicates(subset=['port']).shape

(2828, 2)

In [27]:
port_data = port_data.drop_duplicates(subset=['port'])

Merge

In [28]:
arrivals_19 = pd.merge(arrivals_19, port_data, on=['port'], how='left')

In [29]:
arrivals_19

,port,origin,ShipTypeLevel3,count,GrossTonnage,NetTonnage,country
0,"""BDR$6.)!Y^5;FNC4(G",SAMSUN,General Cargo,1,2598.000000,1163.000000,NaN
1,-,-C?M,Fish Catching,1,721.000000,216.000000,NaN
2,-,KRYM-KAVKAZ,General Cargo,4,57813.192782,30764.864637,NaN
3,-C?M,-,Fish Catching,1,721.000000,216.000000,NaN
4,-TSINGELI,-,General Cargo,1,14453.298196,7691.216159,NaN
...,...,...,...,...,...,...,...
3380,ZONGULDAK,ORDER,General Cargo,1,2160.000000,1243.000000,Turkey
3381,ZONGULDAK,YEISK,General Cargo,1,14453.298196,7691.216159,Turkey
3382,ZONGULDAK TURKEY,TAGANROG RUSSIA,General Cargo,1,6483.000000,3264.000000,NaN
3383,ZPR,KHERSON,General Cargo,2,4125.000000,1486.000000,NaN


In [30]:
arrivals_19['country'].isna().sum()

2716

In [31]:
def check_distance_levenshtein(data: pd.DataFrame, variables=None) -> pd.DataFrame:
    """
    Compute levenshtein distance between list of columns of a Pandas DataFrame
    Parameters
    ----------
    data: Pandas dataframe with some columns to be compared
    variables: List of variables from which distance will be computed. The first one will be used as benchmark
    Returns
    -------
    Initial dataframe with new columns representing levenshtein ratio (computed using :meth:`rapidfuzz.fuzz.partial_ratio`)
    between variables
    """

    if variables is None:
        variables = ['libel_clean_relevanc', 'libel_clean_OFF', 'libel_clean_IRI']

    data[variables] = data[variables].astype(str)
    newvars = ["ratio{}".format(str(i)) for i in range(1, len(variables))]
    data[newvars] = pd.concat(
        [data.apply(lambda x: rapidfuzz.fuzz.partial_ratio(x[variables[0]], x[y]), axis=1) for y in variables[1:]],
        axis=1
    )
    return data

In [32]:
import numpy as np

arrivals_19["port"] = (arrivals_19["port"].replace("None", np.nan)
                        .replace('B 2 A', np.nan)
                        .replace('J0', np.nan)
                        .replace('0', np.nan)
                        .replace(',',np.nan)
                        .replace('G', np.nan)
                        .replace('I0', np.nan)
                        .replace('C', np.nan)
                        .replace('D', np.nan)
                        .replace('C0', np.nan)
                        .replace('S0', np.nan).replace('M', np.nan)
                        .replace('X', np.nan).replace('A', np.nan)
                        .replace('ZRP', np.nan)
                        .replace('AT SEA', np.nan)
                        .replace("$", np.nan)
                        .replace('#HDCR*H!!!)JM0I*!&', np.nan)
                        .replace("%", np.nan)
                        .replace("-", np.nan).replace('-C>-',np.nan)
                        .replace('-C?M',np.nan)
                        .replace('-[C?<)RD\\K$OQ8)',np.nan)
                        .replace( '.',np.nan)
                        .replace( '...',np.nan)
                        .replace('0XA F3L86F,[E^CON',np.nan)
                        .replace('17 B#',np.nan)
                        .replace('2',np.nan)
                        .replace('3',np.nan)
                        .replace('450',np.nan)
                        .replace(r'(\s\/\s[a-zA-Z]+)', '', regex=True)
                        .replace(r'(\/[a-zA-Z]+)', '', regex=True)
                        .replace('FISHING AREA', np.nan))
list_errors = ['AREA3 FISHING GROUND', 'ARM GAURD ON BOARD',
'ARM GUARD ON BOARD', 'ARM.GUARDS ON BOARD', 'ARMED GUARD ON BOARD', 'ARMED GUARDS ON BOAP',
'ARMED GUARDS ON BOAR', 'ARMED GUARDS ONB', 'ARMED GUARDS ONBOARD', 'ARMED ON BOARD',
'ARMGUARD O.B', 'ARMGUARD ONBOARD', 'ARMS GUARDS ON BOARD']
arrivals_19.loc[arrivals_19["port"].isin(list_errors), "port"] = np.nan

In [33]:
arrivals_19

,port,origin,ShipTypeLevel3,count,GrossTonnage,NetTonnage,country
0,"""BDR$6.)!Y^5;FNC4(G",SAMSUN,General Cargo,1,2598.000000,1163.000000,NaN
1,NaN,-C?M,Fish Catching,1,721.000000,216.000000,NaN
2,NaN,KRYM-KAVKAZ,General Cargo,4,57813.192782,30764.864637,NaN
3,NaN,-,Fish Catching,1,721.000000,216.000000,NaN
4,-TSINGELI,-,General Cargo,1,14453.298196,7691.216159,NaN
...,...,...,...,...,...,...,...
3380,ZONGULDAK,ORDER,General Cargo,1,2160.000000,1243.000000,Turkey
3381,ZONGULDAK,YEISK,General Cargo,1,14453.298196,7691.216159,Turkey
3382,ZONGULDAK TURKEY,TAGANROG RUSSIA,General Cargo,1,6483.000000,3264.000000,NaN
3383,ZPR,KHERSON,General Cargo,2,4125.000000,1486.000000,NaN


In [42]:
unique_ports_arrivals = pd.DataFrame(arrivals_19.port.unique())
unique_ports_arrivals.columns = ['port']
cart_prod = unique_ports_arrivals.merge(port_data, how='cross')

In [44]:
cart_prod

,port_x,port_y,country
0,"""BDR$6.)!Y^5;FNC4(G",ST. JAMES HARBOR,United States
1,"""BDR$6.)!Y^5;FNC4(G",MELCHIOR HARBOR,Antarctica
2,"""BDR$6.)!Y^5;FNC4(G",DONGGALA,Indonesia
3,"""BDR$6.)!Y^5;FNC4(G",PSVM TERMINAL,Angola
4,"""BDR$6.)!Y^5;FNC4(G",KOKOPO,Papua New Guinea
...,...,...,...
4264619,_ASPROPYGR0S.,PANJANG,Indonesia
4264620,_ASPROPYGR0S.,IMABARI KO,Japan
4264621,_ASPROPYGR0S.,RADA PUNTA ARENAS,Chile
4264622,_ASPROPYGR0S.,HUELVA,Spain


In [47]:
!pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.3 MB/s eta 0:00:00a 0:00:01


In [48]:
import rapidfuzz
res_fuz_match = check_distance_levenshtein(cart_prod, ["port_x", "port_y"])
res_fuz_match

,port_x,port_y,country,ratio1
0,"""BDR$6.)!Y^5;FNC4(G",ST. JAMES HARBOR,United States,20.000000
1,"""BDR$6.)!Y^5;FNC4(G",MELCHIOR HARBOR,Antarctica,21.052632
2,"""BDR$6.)!Y^5;FNC4(G",DONGGALA,Indonesia,30.769231
3,"""BDR$6.)!Y^5;FNC4(G",PSVM TERMINAL,Angola,15.384615
4,"""BDR$6.)!Y^5;FNC4(G",KOKOPO,Papua New Guinea,0.000000
...,...,...,...,...
4264619,_ASPROPYGR0S.,PANJANG,Indonesia,28.571429
4264620,_ASPROPYGR0S.,IMABARI KO,Japan,37.500000
4264621,_ASPROPYGR0S.,RADA PUNTA ARENAS,Chile,31.578947
4264622,_ASPROPYGR0S.,HUELVA,Spain,25.000000


In [59]:
idx = res_fuz_match.groupby(['port_x'])['ratio1'].transform(max) == res_fuz_match['ratio1']
res_fuz_match2 = res_fuz_match[idx]
res_fuz_match2 = res_fuz_match2.groupby(['port_x']).first()
res_fuz_match2["true_match"] = np.nan
res_fuz_match2

,port_y,country,ratio1,true_match
port_x,,,,
"""BDR$6.)!Y^5;FNC4(G",BAR,Montenegro,66.666667,NaN
-TSINGELI,TSINGELI,Greece,100.000000,NaN
451,KOME KRIBI 1 MARINE TERMINAL,Cameroon,33.333333,NaN
451 CAZ,CADIZ,Spain,75.000000,NaN
451 CLBLAX80!>5,BULA,Indonesia,75.000000,NaN
...,...,...,...,...
ZONGULDAK,ZONGULDAK,Turkey,100.000000,NaN
ZONGULDAK TURKEY,ZONGULDAK,Turkey,100.000000,NaN
ZPR,PRINCE OLAV HARBOR,South Georgia and South Sandwich Islands,80.000000,NaN


In [60]:
res_fuz_match2.loc[res_fuz_match2["ratio1"] > 80, "true_match"] = \
    res_fuz_match2.loc[res_fuz_match2["ratio1"] > 80, "port_y"]

In [61]:
res_fuz_match2.head(30)

,port_y,country,ratio1,true_match
port_x,,,,
"""BDR$6.)!Y^5;FNC4(G",BAR,Montenegro,66.666667,NaN
-TSINGELI,TSINGELI,Greece,100.000000,TSINGELI
451,KOME KRIBI 1 MARINE TERMINAL,Cameroon,33.333333,NaN
451 CAZ,CADIZ,Spain,75.000000,NaN
451 CLBLAX80!>5,BULA,Indonesia,75.000000,NaN
451 Y DISTANCE,NICE,France,85.714286,NICE
92KM,KEMI,Finland,66.666667,NaN
> UA YUZ,AYR,United Kingdom,66.666667,NaN
>I Z M A I L > A,KUMAI,Indonesia,60.000000,NaN


In [63]:
res_fuz_match2['true_match'].isna().sum()

683

In [65]:
res_fuz_match2.shape

(1508, 4)

In [66]:
res_fuz_match2.loc[res_fuz_match2["ratio1"] < 80, :].head(30)

,port_y,country,ratio1,true_match
port_x,,,,
"""BDR$6.)!Y^5;FNC4(G",BAR,Montenegro,66.666667,NaN
451,KOME KRIBI 1 MARINE TERMINAL,Cameroon,33.333333,NaN
451 CAZ,CADIZ,Spain,75.000000,NaN
451 CLBLAX80!>5,BULA,Indonesia,75.000000,NaN
92KM,KEMI,Finland,66.666667,NaN
> UA YUZ,AYR,United Kingdom,66.666667,NaN
>I Z M A I L > A,KUMAI,Indonesia,60.000000,NaN
A=>GR AST,VASTO,Italy,75.000000,NaN
AA 455,APIA,Samoa,66.666667,NaN
